# Features extraction

Turn raw data into features.

## Extract URL features

The following features are extracted from the URL:

- **URLLength**: Number of characters in the URL.
- **Domain**: Domain name extracted from the URL.
- **DomainLength**: Number of characters in the domain name.
- **IsDomainIP**: Indicates if the domain name is an IP address.
- **TLD**: TLD (Top Level Domain) is the last part of the domain name, such as .com or .edu.
- **URLSimilarityIndex**:
- **CharContinuationRate**: Ratio of the number of continuous characters in the URL.
- **TLDLegitimateProb**:
- **URLCharProb**:
- **TLDLength**: Number of characters in the TLD.
- **NoOfSubDomain**: Number of subdomains in the URL.
- **HasObfuscation**: Indicates if the URL has obfuscated characters like %20, %4D, etc.
- **NoOfObfuscatedChar**: Number of obfuscated characters in the URL.
- **ObfuscationRatio**: Ratio of obfuscated characters in the URL.
- **NoOfLettersInURL**: Number of letters in the URL.
- **LetterRatioInURL**: Ratio of letters in the URL.
- **NoOfDegitsInURL**: Number of digits in the URL.
- **DegitRatioInURL**: Ratio of digits in the URL.
- **NoOfEqualsInURL**: Number of equal signs (=) in the URL.
- **NoOfQMarkInURL**: Number of question marks (?) in the URL.
- **NoOfAmpersandInURL**: Number of ampersands (&) in the URL.
- **NoOfOtherSpecialCharsInURL**: Number of special characters other than equals, question marks, and ampersands in the URL.
- **SpacialCharRatioInURL**: Ratio of all special characters in the URL. A special character is any character that is not a letter or a digit.
- **IsHTTPS**: Indicates if the webpage is running on unsecured HTTP (hypertext transfer protocol) or secured HTTPS.

**Notes**:

- We need to differentiate between URL and domain. For example, in the URL `https://www.google.com/search?q=python`, the domain is `www.google.com`. This is commonly referred to as the hostname. The domain is also commonly referred to as the base domain or the root domain (google.com). But, in our case, we'll refer to the hostname as the domain (which is ultimetly not incorrect).
- Boolean features are converted to numerical values (0=False; 1=True).

### Import libraries

In [1]:
from urllib.parse import urlparse, unquote
import re

### Extract features

In [2]:
def IsDomainIP(domain):
    # This regex will match any sequence of four numbers separated by dots. This is a simple way to check if a string is an IP address.
    # However, it doesn't strictly validate IP addresses. For example, it will match 999.999.999.999, which is not a valid IP address.
    ip_pattern = r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}'
    is_ip = bool(re.search(ip_pattern, domain))
    return int(is_ip)

print(IsDomainIP('www.google.com')) # 0
print(IsDomainIP('192.168.1.1')) # 1
print(IsDomainIP('999.999.999.999')) # 1 (True, but it's not a valid IP address)

0
1
1


In [3]:
def NoOfSubDomain(domain):
    # IP addresses are not domain names, thus they don't have subdomains. 
    # Subdomains are part of the DNS hierarchy and are only used in domain names.
    if (IsDomainIP(domain)):
        return 0
    
    domains = domain.split('.')
    return len(domains) - 2 # Subtract apex domain and TLD

print(NoOfSubDomain('docs.python.org')) # 1
print(NoOfSubDomain('google.com')) # 0
print(NoOfSubDomain('192.168.1.1')) # 0

1
0
0


In [4]:
# https://github.com/arvindbitm/PhiUSIIL/blob/main/CharConRate.ipynb
# They are stripping 'www' and the TLD from the domain name before calculating the character continuation rate.
# Why only 'www'? What about other subdomains?

def CharConRate(url):
    rate=0 # This variable is not even used 🤷‍♂️
    ln=len(url)
    chC, nmC, spC = 0,0,0
    maxCh, maxNm, MaxSp = 0,0,0
    for i in range(0,ln):
        ch = url[i]
        if ch.isalpha():
            chC=chC+1
            if(nmC>0):
                if(maxNm<nmC):
                    maxNm=nmC
                    nmC=0
            elif(spC>0):
                if(MaxSp<spC):
                    MaxSp=spC
                    spC=0
            nmC,spC=0,0
                
        elif ch.isdigit():
            nmC=nmC+1
            if(chC>0):
                if(maxCh<chC):
                    maxCh=chC
                    chC=0
            elif(spC>0):
                if(MaxSp<spC):
                    MaxSp=spC
                    spC=0
            chC,spC=0,0
        else:
            spC=spC+1
            if(nmC>0):
                if(maxNm<nmC):
                    maxNm=nmC
                    nmC=0
            elif(chC>0):
                if(maxCh<chC):
                    maxCh=chC
                    chC=0
            nmC,chC=0,0
    
    if(maxCh<chC):
        maxCh=chC
    if(maxNm<nmC):
        maxNm=nmC
    if(MaxSp<spC):
        MaxSp=spC
    return (maxCh+maxNm+MaxSp)/ln

In [5]:
def HasObfuscation(str):
    decoded_str = unquote(str)
    return int(decoded_str != str)

print(HasObfuscation('https://facebook.com'))
print(HasObfuscation('https://facebook.com@%61%62%63.%43%4F%4D'))

0
1


In [6]:
def NoOfObfuscatedChar(str):
    # Regular expression to find percent-encoded characters.
    # A percent-encoded character is a character that is represented by a percent sign followed by two hexadecimal digits.
    encoded_char_pattern = r'%[0-9A-Fa-f]{2}'
    # Find all matches of the pattern in the URL
    encoded_chars = re.findall(encoded_char_pattern, str)
    return len(encoded_chars)

print(NoOfObfuscatedChar('https://facebook.com'))
print(NoOfObfuscatedChar('https://facebook.com@%61%62%63.%43%4F%4D'))

0
6


In [7]:
def NoOfInURL(str, pattern):
    """
    Counts the number of occurrences of a given pattern in a string.

    Parameters:
    - str (str): The input string to search for matches.
    - pattern (str): The pattern to search for in the input string.

    Returns:
    - int: The number of matches found.

    Example:
    >>> NoOfInURL("https://facebook.com?param=value", r"=")
    1
    """
    # Find all matches of the pattern in the URL
    matches = re.findall(pattern, str)
    return len(matches)

print(NoOfInURL('https://facebook.com', r'\?'))
print(NoOfInURL('https://facebook.com?param=value', r'\?'))

0
1


In [8]:
# Refering to 'URL' but seems to be calculating for 'Domain' on the dataset
# Same applies to the all ...InURL calculations
def SpecialCharRatioInURL(url):
    # Negative lookahead to match any character that is not a letter or a digit
    special_char_pattern = r'[^A-Za-z0-9]'
    special_chars = re.findall(special_char_pattern, url)
    return len(special_chars) / len(url)

print(SpecialCharRatioInURL('facebook.com'))
print(SpecialCharRatioInURL('facebook.com?param=value'))

0.08333333333333333
0.125


In [9]:
# Even though most URLs in the dataset do not contains query params, those kind of URLs are more realistic.
test_url = 'https://www.google.com/search?q=alan+turing'

def URLFeatures(url):
    parsed_url = urlparse(url)
    domain = parsed_url.hostname
    tld = domain.split('.')[-1]

    return {
        'URLLength': len(url),
        'Domain': domain,
        'DomainLength': len(domain),
        'IsDomainIP': IsDomainIP(domain),
        'TLD': tld,
        'TLDLength': len(tld),
        'TLDLegitimateProb': '',
        'URLSimilarityIndex': '',
        'CharContinuationRate': CharConRate(domain.split('.')[1 if NoOfSubDomain('google.com') > 0 else 0]), # For consistency, we are stripping the first subdomain (not just 'www') and the TLD
        'URLCharProb': '',
        'NoOfSubDomain': NoOfSubDomain(domain),
        'HasObfuscation': HasObfuscation(domain),
        'NoOfObfuscatedChar': NoOfObfuscatedChar(domain),
        'ObfuscationRatio': NoOfObfuscatedChar(domain) / len(domain),
        'NoOfLettersInURL': NoOfInURL(url, r'[a-zA-Z]'),
        'LetterRatioInURL': NoOfInURL(url, r'[a-zA-Z]') / len(url),
        'NoOfDegitsInURL': NoOfInURL(url, r'\d'),
        'DegitRatioInURL': NoOfInURL(url, r'\d') / len(url),
        'NoOfEqualsInURL': NoOfInURL(url, r'='),
        'NoOfQMarkInURL': NoOfInURL(url, r'\?'),
        'NoOfAmpersandInURL': NoOfInURL(url, r'&'),
        'NoOfOtherSpecialCharsInURL': NoOfInURL(url, r'[^a-zA-Z\d=&\?]'),
        'SpacialCharRatioInURL': SpecialCharRatioInURL(url), # I guess this is a typo and it should be 'SpecialCharRatioInURL'
        'IsHTTPS': int(parsed_url.scheme == 'https'),
    }

URLFeatures(test_url)

{'URLLength': 43,
 'Domain': 'www.google.com',
 'DomainLength': 14,
 'IsDomainIP': 0,
 'TLD': 'com',
 'TLDLength': 3,
 'TLDLegitimateProb': '',
 'URLSimilarityIndex': '',
 'CharContinuationRate': 1.0,
 'URLCharProb': '',
 'NoOfSubDomain': 1,
 'HasObfuscation': 0,
 'NoOfObfuscatedChar': 0,
 'ObfuscationRatio': 0.0,
 'NoOfLettersInURL': 34,
 'LetterRatioInURL': 0.7906976744186046,
 'NoOfDegitsInURL': 0,
 'DegitRatioInURL': 0.0,
 'NoOfEqualsInURL': 1,
 'NoOfQMarkInURL': 1,
 'NoOfAmpersandInURL': 0,
 'NoOfOtherSpecialCharsInURL': 7,
 'SpacialCharRatioInURL': 0.20930232558139536,
 'IsHTTPS': 1}